Allow the two machines to play for n times. One agent has weights and features and calculates the value and makes the move while the other agent makes random moves. create a dataframe with the moves made and the evaluation score. Once the playing is complete, the actual Vtrain values are populated in the dataframe. If it is the last move (game over move), the Vtrain gets 100,0,-100 (w,d,l). Otherwise, the Vtrain value is the evaluation score (estimate) of the subsequent board state. For instance, the Vtrain value of the last but one board state will be the Vestimate of the last move.

In [32]:
import numpy as np
import pandas as pd
import os
import random
#from random import randint
from IPython.display import display, clear_output

## The below code is for feature extraction

In [2]:
def evalboard(test_board):
    #print("Test board inside evalboard", test_board)
    ## two squares in a row (can be row, column or diagonal) are filled with 1 and the other square is empty
    f1 = []
    
    if((np.count_nonzero(test_board[0,:])==2) & (np.sum(test_board[0,:])==2)):
        f1.append(1)
    if((np.count_nonzero(test_board[1,:])==2) & (np.sum(test_board[1,:])==2)):
        f1.append(1)
    if((np.count_nonzero(test_board[2,:])==2) & (np.sum(test_board[2,:])==2)):
        f1.append(1)
    if((np.count_nonzero(test_board[:,0])==2) & (np.sum(test_board[:,0])==2)):
        f1.append(1)
    if((np.count_nonzero(test_board[:,1])==2) & (np.sum(test_board[:,1])==2)):
        f1.append(1)
    if((np.count_nonzero(test_board[:,2])==2) & (np.sum(test_board[:,2])==2)):
        f1.append(1)
    if((np.count_nonzero(test_board.diagonal())==2) & (np.trace(test_board)==2)):
        f1.append(1)
    if((np.count_nonzero(np.fliplr(test_board).diagonal())==2) & (np.trace(np.fliplr(test_board))==2)):
        f1.append(1)

    
    
    ## two squares in a row (can be row, column or diagonal) are filled with 2 and the other square is empty
    f2 = []

    if((np.count_nonzero(test_board[0,:])==2) & (np.sum(test_board[0,:])==4)):
        f2.append(2)
    if((np.count_nonzero(test_board[1,:])==2) & (np.sum(test_board[1,:])==4)):
        f2.append(2)
    if((np.count_nonzero(test_board[2,:])==2) & (np.sum(test_board[2,:])==4)):
        f2.append(2)
    if((np.count_nonzero(test_board[:,0])==2) & (np.sum(test_board[:,0])==4)):
        f2.append(2)
    if((np.count_nonzero(test_board[:,1])==2) & (np.sum(test_board[:,1])==4)):
        f2.append(2)
    if((np.count_nonzero(test_board[:,2])==2) & (np.sum(test_board[:,2])==4)):
        f2.append(2)
    if((np.count_nonzero(test_board.diagonal())==2) & (np.trace(test_board)==4)):
        f2.append(2)
    if((np.count_nonzero(np.fliplr(test_board).diagonal())==2) & (np.trace(np.fliplr(test_board))==4)):
        f2.append(2)
    
    ## One 2 in an otherwise empty row (column, row or diagonal)
    f4 = []
    
    if((np.count_nonzero(test_board[0,:])==1) & (np.sum(test_board[0,:])==2)):
        f4.append(2)
    if((np.count_nonzero(test_board[1,:])==1) & (np.sum(test_board[1,:])==2)):
        f4.append(2)
    if((np.count_nonzero(test_board[2,:])==1) & (np.sum(test_board[2,:])==2)):
        f4.append(2)
    if((np.count_nonzero(test_board[:,0])==1) & (np.sum(test_board[:,0])==2)):
        f4.append(2)
    if((np.count_nonzero(test_board[:,1])==1) & (np.sum(test_board[:,1])==2)):
        f4.append(2)
    if((np.count_nonzero(test_board[:,2])==1) & (np.sum(test_board[:,2])==2)):
        f4.append(2)
    if((np.count_nonzero(test_board.diagonal())==1) & (np.trace(test_board)==2)):
        f4.append(2)
    if((np.count_nonzero(np.fliplr(test_board).diagonal())==1) & (np.trace(np.fliplr(test_board))==2)):
        f4.append(2)
        
    ## One 1 in an otherwise empty row (column, row or diagonal)
    f3 = []
    
    if(np.sum(test_board[0,:])==1):
        f3.append(1)
        
    if(np.sum(test_board[1,:])==1):
        f3.append(1)
        
    if(np.sum(test_board[2,:])==1):
        f3.append(1)
        
    if(np.sum(test_board[:,0])==1):
        f3.append(1)
        
    if(np.sum(test_board[:,1])==1):
        f3.append(1)
        
    if(np.sum(test_board[:,2])==1):
        f3.append(1)
        
    if(np.trace(test_board)==1):
        f3.append(1)
       
    if(np.trace(np.fliplr(test_board))==1):
        f3.append(1)
       

    

    ## Three cells in a row is filled with 1
    f5 = []
    
    if(test_board[0,:]==1).all():
        f5.append(1)
    if(test_board[1,:]==1).all():
        f5.append(1)
    if(test_board[2,:]==1).all():
        f5.append(1)
    if(test_board.diagonal()==1).all():
        f5.append(1)
    if(np.fliplr(test_board).diagonal()==1).all():
        f5.append(1)

    
    ## Three cells in a row is filled with 2
    f6 = []

    
    if(test_board[0,:]==2).all():
        f6.append(2)
    if(test_board[1,:]==2).all():
        f6.append(2)
    if(test_board[2,:]==2).all():
        f6.append(2)
    if(test_board.diagonal()==2).all():
        f6.append(2)
    if(np.fliplr(test_board).diagonal()==2).all():
        f6.append(2)
    
    return(len(f1),len(f2),len(f3),len(f4),len(f5),len(f6))

### The below function is for making the moves of the machine which is trained to become smart

In [3]:
def machine1_move(old_board_state,w0,w1,w2,w3,w4,w5,w6):
    feature_list = []
    board_state = np.copy(old_board_state)
    x, y = np.where(board_state==0)
    #print("x,y are",x,y)
    evalscore = []
    for i in range(len(x)):
        board_state[x[i],y[i]]=1

        ft1, ft2, ft3, ft4, ft5, ft6 = evalboard(board_state)
        feature_list.append([ft1,ft2,ft3,ft4,ft5,ft6])
        #
        evalscore.append(w0 + w1*ft1 + w2*ft2 + w3*ft3 + w4*ft4 + w5*ft5 + w6*ft6)
        feature_list[i].extend([w0 + w1*ft1 + w2*ft2 + w3*ft3 + w4*ft4 + w5*ft5 + w6*ft6])
        #print("features",feature_list)
        board_state = np.copy(old_board_state)
        

    position_index = np.argmax(np.asarray(evalscore))
    optimal_feature = feature_list[position_index]
    #print("Optimal feature is ", optimal_feature)
    #print("position_index is ",position_index)

    machine1_position = x[position_index]*3 + y[position_index]
    #moves.append(machine1_position)
    
    return(machine1_position, max(evalscore), optimal_feature)




### the below code is for the machine which doesnt learn - makes random moves 

In [4]:

def machine2_move(game_moves):
    while(True):
        machine2_position = randint(0,8)
        if machine2_position not in game_moves:
            #game_board[machine2_position // 3, machine2_position % 3] = 2
            #moves.append(machine2_position)
            
            #clear_output()
            print("Machine 2's move")
            break
    return(machine2_position)



### Below code is for updating weights after each training game is played

In [5]:
def weight_update(X, y, game_counter):
    print("Game counter", game_counter)
    f1 = [sublist[0] for sublist in X]
    f2 = [sublist[1] for sublist in X]
    f3 = [sublist[2] for sublist in X]
    f4 = [sublist[3] for sublist in X]
    f5 = [sublist[4] for sublist in X]
    f6 = [sublist[5] for sublist in X]
    Vestimate = [sublist[6] for sublist in X]
    Vtrain_flattened = [item for sublist in y for item in sublist]
    lr = 0.1
    print("length of f1",len(f1))
    for i in range(0, len(f1)):
        w0_new = w0 + lr*(Vtrain_flattened[i] - Vestimate[i])
        w1_new = w1 + lr*(Vtrain_flattened[i] - Vestimate[i])*f1[i]
        w2_new = w2 + lr*(Vtrain_flattened[i] - Vestimate[i])*f2[i]
        w3_new = w3 + lr*(Vtrain_flattened[i] - Vestimate[i])*f3[i]
        w4_new = w4 + lr*(Vtrain_flattened[i] - Vestimate[i])*f4[i]
        w5_new = w5 + lr*(Vtrain_flattened[i] - Vestimate[i])*f5[i]
        w6_new = w6 + lr*(Vtrain_flattened[i] - Vestimate[i])*f6[i]
        print(i)
        if i==(len(f1)-1):
            break

        Vestimate[i+1] = w0_new + w1_new*f1[i+1] + w2_new*f2[i+1] + w3_new*f3[i+1] + w4_new*f4[i+1] + w5_new*f5[i+1] + w6_new*f6[i+1]
    print(w0_new,w1_new,w2_new,w3_new,w4_new,w5_new,w6_new)
    return(w0_new,w1_new,w2_new,w3_new,w4_new,w5_new,w6_new)




### Below code is for the whole training process

In [48]:
##List of winning moves - whenever any combination in this achieved it is game over
winning_moves = [[0,1,2],[0,3,6],[0,4,8],[1,4,7],[2,5,8],[2,4,6],[3,4,5],[6,7,8]]
positions = []
start_index = 0
game_feature = []

Vtrain = []

## Initializing weights for the learning. This can be made random also instead of a fixed 0.5
w0 = w1 = w2 = w3 = w4 = w5 = w6 = random.uniform(0,1)
print("intial wights",w0,w1,w2,w3,w4,w5,w6)


for m in range(0,100):
    
    ##Initializing different lists - moves contain overall moves made by both the agents
    ## This is to prevent repetition of moves
    ## The individual agent moves are to compare with the winning moves list
    ## The board is initialized with zeros
    
    moves = []
    machine1_moves = []
    machine2_moves = []
    Vest = []
    feature_list = []
    game_feature = []
    Vtrain=[]
    game_board = np.zeros([3,3])
    print(game_board)
    
    ## This loop is for the game play - a maximum of 9 moves can be made in a game
    for i in range(0,9):
        #game_feature = []
        ## Initializing a game over flag to 0
        GO = 0
        
        ## Calling the function machine1_move for making the moves of the agent 1
        ## and receiving the move position in machine1_position variable
        ## Passing the weights as function arguments 
        machine1_position, vestimate, feature_list = machine1_move(game_board, w0, w1, w2, w3, w4, w5, w6)
        print("Vestimate is ", vestimate)
        Vest.append(vestimate)
        
        ##Updating the game board with the machine1 move position
        game_board[machine1_position // 3, machine1_position % 3] = 1
        print("Machine 1's move")
        print(game_board)
        
        ## Creating a copy of game board into mc1
        ## Pushing the game board status into the positions list
        ## this will useful later when creating the dataframe of game board positions
        
        #mc1 = np.copy(game_board)
        #positions.append(mc1.ravel().tolist())
        game_feature.append(feature_list)
 
        
        ## Pushing the moves machine 1 into the moves list and the machine1_moves list
        moves.append(machine1_position)
        print("Moves array",moves)
        machine1_moves.append(machine1_position)
        machine1_counter = 0
        
        ## If more than two moves are made, then keep checking in the winning moves list
        if len(machine1_moves) > 2:
            for j in range(len(winning_moves)):
                #human_counter = 0
                ## For each set (3 positions) of winning moves, the values in the machine1_moves
                ## are checked with the winning moves set - and if there is a match of 3, the game is 
                ## declared won by machine 1
                machine1_counter = [a for a in machine1_moves if a in winning_moves[j]]
                if len(machine1_counter)==3:
                    print("Game Over")
                    print("Machine1 won")
                    GO = 1
                    break
        
        ## If game over, the GO flag will be set to 1 in the previous code block and the previous loop is exited.
        ## Now, for all the gameboard arrays in the position list (which includes moves by machine 2 as well)
        ## a 1 is added - to indicate that the game was won by agent 1.
        ## Else if the number of moves is 9 without the GO flag set to 1, then it is a tie game and hence '0' is 
        ## added to the position arrays
        if GO == 1:
            Vest.append(100)
            Vest.pop(0)
            Vtrain.append(Vest)

            break
            
        elif len(machine1_moves)+len(machine2_moves) == 9:
            print("Game Tie")
            Vest.append(0)
            Vest.pop(0)
            Vtrain.append(Vest)
            break
            
            
        ## Calling the function machine2_move for making the moves of the agent 2
        ## and receiving the move position in machine2_position variable
        
        
        machine2_position = machine2_move(moves)
        
        game_board[machine2_position // 3, machine2_position % 3] = 2
        print(game_board)
        
        mc2 = np.copy(game_board)
        #positions.append(mc2.ravel().tolist())
        
        moves.append(machine2_position)
        machine2_moves.append(machine2_position)
        machine2_counter = 0
        
        if len(machine2_moves) > 2:
            for h in range(len(winning_moves)):
                #machine_counter = 0
                machine2_counter = [b for b in machine2_moves if b in winning_moves[h]]
                if len(machine2_counter)==3:
                    print("Game Over")
                    print("Machine2 won")
                    GO = 2
                    break

        if GO == 2:
            Vest.append(-100)
            Vest.pop(0)
            Vtrain.append(Vest)
            break
    #print("GAME FEATURE",game_feature)
    w0,w1,w2,w3,w4,w5,w6 = weight_update(game_feature, Vtrain,m)

intial wights 0.7957345585212187 0.7957345585212187 0.7957345585212187 0.7957345585212187 0.7957345585212187 0.7957345585212187 0.7957345585212187
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
Vestimate is  3.978672792606093
Machine 1's move
[[0. 0. 0.]
 [0. 1. 0.]
 [0. 0. 0.]]
Moves array [4]
Machine 2's move
[[0. 0. 0.]
 [0. 1. 0.]
 [2. 0. 0.]]
Vestimate is  6.365876468169749
Machine 1's move
[[0. 0. 1.]
 [0. 1. 0.]
 [2. 0. 0.]]
Moves array [4, 6, 2]
Machine 2's move
[[0. 2. 1.]
 [0. 1. 0.]
 [2. 0. 0.]]
Vestimate is  4.774407351127312
Machine 1's move
[[0. 2. 1.]
 [0. 1. 1.]
 [2. 0. 0.]]
Moves array [4, 6, 2, 1, 5]
Machine 2's move
[[0. 2. 1.]
 [0. 1. 1.]
 [2. 0. 2.]]
Vestimate is  2.387203675563656
Machine 1's move
[[1. 2. 1.]
 [0. 1. 1.]
 [2. 0. 2.]]
Moves array [4, 6, 2, 1, 5, 8, 0]
Machine 2's move
[[1. 2. 1.]
 [0. 1. 1.]
 [2. 2. 2.]]
Game Over
Machine2 won
Game counter 0
length of f1 4
0
1
2
3
-9.569507603840023 -9.569507603840023 -9.569507603840023 0.7957345585212187 0.7957345585212187 

Machine 2's move
[[2. 1. 0.]
 [1. 0. 0.]
 [2. 0. 0.]]
Vestimate is  -11.8303808917698
Machine 1's move
[[2. 1. 0.]
 [1. 0. 0.]
 [2. 0. 1.]]
Moves array [1, 0, 3, 6, 8]
Machine 2's move
[[2. 1. 0.]
 [1. 2. 0.]
 [2. 0. 1.]]
Vestimate is  22.97977212536735
Machine 1's move
[[2. 1. 0.]
 [1. 2. 0.]
 [2. 1. 1.]]
Moves array [1, 0, 3, 6, 8, 4, 7]
Machine 2's move
[[2. 1. 2.]
 [1. 2. 0.]
 [2. 1. 1.]]
Game Over
Machine2 won
Game counter 23
length of f1 4
0
1
2
3
4.98159671537411 -29.854461403838798 -1.3440342632210136 -22.27232858958053 -5.481042855462032 27.718374821224693 0.7957345585212187
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
Vestimate is  -39.56306046378695
Machine 1's move
[[0. 1. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
Moves array [1]
Machine 2's move
[[0. 1. 0.]
 [0. 0. 0.]
 [0. 0. 2.]]
Vestimate is  -58.10727898896928
Machine 1's move
[[0. 1. 0.]
 [0. 0. 0.]
 [0. 1. 2.]]
Moves array [1, 8, 7]
Machine 2's move
[[0. 1. 0.]
 [0. 0. 0.]
 [2. 1. 2.]]
Vestimate is  -50.52514617471101
Machine 1's move
[[

 [2. 0. 0.]]
Vestimate is  401.7735734586564
Machine 1's move
[[0. 0. 1.]
 [0. 1. 0.]
 [2. 0. 0.]]
Moves array [4, 6, 2]
Machine 2's move
[[0. 0. 1.]
 [0. 1. 0.]
 [2. 0. 2.]]
Vestimate is  116.53047932842469
Machine 1's move
[[0. 0. 1.]
 [0. 1. 0.]
 [2. 1. 2.]]
Moves array [4, 6, 2, 8, 7]
Machine 2's move
[[0. 0. 1.]
 [0. 1. 2.]
 [2. 1. 2.]]
Vestimate is  26.04714681693391
Machine 1's move
[[0. 1. 1.]
 [0. 1. 2.]
 [2. 1. 2.]]
Moves array [4, 6, 2, 8, 7, 5, 1]
Game Over
Machine1 won
Game counter 38
length of f1 4
0
1
2
3
1.695562012611422 -88.87022447631863 -0.436308191174005 45.241666255745386 60.647870886676984 1.3878170613037248 0.7957345585212187
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
Vestimate is  182.66222703559296
Machine 1's move
[[0. 0. 0.]
 [0. 1. 0.]
 [0. 0. 0.]]
Moves array [4]
Machine 2's move
[[0. 0. 2.]
 [0. 1. 0.]
 [0. 0. 0.]]
Vestimate is  349.1996350646923
Machine 1's move
[[0. 0. 2.]
 [0. 1. 0.]
 [1. 0. 0.]]
Moves array [4, 2, 6]
Machine 2's move
[[0. 0. 2.]
 [0. 1. 0.]

-232.0306122905053 -776.2194578204558 -379.2101288581706 274.82654440371476 310.43354913196714 -138.236540901461 0.7957345585212187
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
Vestimate is  867.2755653243537
Machine 1's move
[[0. 0. 0.]
 [0. 1. 0.]
 [0. 0. 0.]]
Moves array [4]
Machine 2's move
[[2. 0. 0.]
 [0. 1. 0.]
 [0. 0. 0.]]
Vestimate is  1762.9692079920028
Machine 1's move
[[2. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
Moves array [4, 0, 8]
Machine 2's move
[[2. 0. 0.]
 [0. 1. 2.]
 [0. 0. 1.]]
Vestimate is  902.8825700526061
Machine 1's move
[[2. 0. 0.]
 [1. 1. 2.]
 [0. 0. 1.]]
Moves array [4, 0, 8, 5, 3]
Machine 2's move
[[2. 2. 0.]
 [1. 1. 2.]
 [0. 0. 1.]]
Vestimate is  -733.4235257072464
Machine 1's move
[[2. 2. 1.]
 [1. 1. 2.]
 [0. 0. 1.]]
Moves array [4, 0, 8, 5, 3, 1, 2]
Machine 2's move
[[2. 2. 1.]
 [1. 1. 2.]
 [2. 0. 1.]]
Vestimate is  -232.0306122905053
Machine 1's move
[[2. 2. 1.]
 [1. 1. 2.]
 [2. 1. 1.]]
Moves array [4, 0, 8, 5, 3, 1, 2, 6, 7]
Game Tie
Game counter 58
length of f1 5
0


Machine 2's move
[[0. 0. 0.]
 [0. 1. 0.]
 [2. 0. 0.]]
Vestimate is  608.2333968218661
Machine 1's move
[[0. 0. 1.]
 [0. 1. 0.]
 [2. 0. 0.]]
Moves array [4, 6, 2]
Machine 2's move
[[0. 0. 1.]
 [0. 1. 0.]
 [2. 0. 2.]]
Vestimate is  -951.9524984630552
Machine 1's move
[[0. 0. 1.]
 [0. 1. 0.]
 [2. 1. 2.]]
Moves array [4, 6, 2, 8, 7]
Machine 2's move
[[0. 2. 1.]
 [0. 1. 0.]
 [2. 1. 2.]]
Vestimate is  -143.01959767072375
Machine 1's move
[[1. 2. 1.]
 [0. 1. 0.]
 [2. 1. 2.]]
Moves array [4, 6, 2, 8, 7, 1, 0]
Machine 2's move
[[1. 2. 1.]
 [2. 1. 0.]
 [2. 1. 2.]]
Vestimate is  -185.35394180023405
Machine 1's move
[[1. 2. 1.]
 [2. 1. 1.]
 [2. 1. 2.]]
Moves array [4, 6, 2, 8, 7, 1, 0, 3, 5]
Game Tie
Game counter 76
length of f1 5
0
1
2
3
4
-152.7710732952951 -1142.2250539091162 -611.7023291323751 42.3343441295103 290.95780898727435 -13.49372759623014 0.7957345585212187
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
Vestimate is  16.566303222746086
Machine 1's move
[[0. 0. 0.]
 [0. 1. 0.]
 [0. 0. 0.]]
Moves

## Execute below two blocks for playing / testing with human player

##### This is for making the moves of the human player

In [7]:
def human_move(moves_entered):
    move_entered = 0
    while (move_entered == 0):
        human_position = int(input("enter your move 1 - 9 ")) - 1
        if human_position > 8 or human_position < 0:
            print("Enter your move between 1 - 9")
        elif human_position not in moves_entered:
            #game_board[position // 3, position % 3] = 1
            move_entered = 2
            #clear_output()
            #print(game_board)
        else:
            print("Enter into a blank cell")
    return(human_position)

#### Pitting the human with the smart agent

In [55]:

#positions = []
winning_moves = [[0,1,2],[0,3,6],[0,4,8],[1,4,7],[2,5,8],[2,4,6],[3,4,5],[6,7,8]]
#start_index = 0


real_game_moves = []
machine_moves = []
human_moves = []
real_game_board = np.zeros([3,3])
print(real_game_board)


for i in range(0,9):
    GO = 0
    
    gb_copy = np.copy(real_game_board)
    
    machine_position, vestimate, feature_list = machine1_move(real_game_board, w0, w1, w2, w3, w4, w5, w6)
       
    #machine_position = machine_move(gb_copy.ravel())
    #print(machine_position)
    real_game_board[machine_position // 3, machine_position % 3] = 1
    clear_output()
    print(real_game_board)
    real_game_moves.append(machine_position)
    machine_moves.append(machine_position)
    
    if len(machine_moves) > 2:
        for j in range(len(winning_moves)):
            
            machine_counter = [a for a in machine_moves if a in winning_moves[j]]
            if len(machine_counter)==3:
                print("Game Over")
                print("Machine won")
                GO = 1
                break
    
    if GO == 1:
        break
            
    elif len(machine_moves)+len(human_moves) == 9:
        print("Game Tie")
        break
      
    
    human_game_position = human_move(real_game_moves)
    real_game_board[human_game_position // 3, human_game_position % 3] = 2
    clear_output()
    print(real_game_board)
    real_game_moves.append(human_game_position)
    human_moves.append(human_game_position)
    
    if len(human_moves) > 2:
        for h in range(len(winning_moves)):
            #machine_counter = 0
            human_counter = [b for b in human_moves if b in winning_moves[h]]
            if len(human_counter)==3:
                print("Game Over")
                print("Human won")
                GO = 1
                break
    
    if GO == 1:
        break   
    
    

[[2. 2. 2.]
 [0. 1. 0.]
 [0. 1. 1.]]
Game Over
Human won
